In [1]:
import requests
import csv
from datetime import datetime
def fetch_shopee_comments(item_id, shop_id, limit=10, offset=0):
    """
    Lấy dữ liệu bình luận từ API của Shopee
    """
    url = "https://shopee.vn/api/v4/item/get_ratings"
    params = {
        "itemid": item_id,
        "shopid": shop_id,
        "limit": limit,
        "offset": offset,
        "filter": 0,  # 0: tất cả, 1: có hình ảnh, 2: 5 sao, 3: 4 sao, ...
        "type": 0     # 0: bình thường
    }
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        comments = data.get("data", {}).get("ratings", [])
        return comments
    else:
        print(f"Error: {response.status_code}")
        return []

def save_comments_to_csv(comments, file_path="shopee_data_raw.csv"):
    """
    Lưu danh sách bình luận vào file CSV với đường dẫn tùy chỉnh
    """
    with open(file_path, mode="w", newline="", encoding="utf-8-sig") as file:
        writer = csv.writer(file, delimiter=';')  # Dùng ';' để tương thích với Excel
        # Ghi header
        writer.writerow(["Rating", "Comment", "Order ID", "Time"])

        # Ghi dữ liệu
        for comment in comments:
            if not isinstance(comment, dict):
                print(f"Warning: Unexpected data format: {comment}")
                continue  # Bỏ qua nếu dữ liệu không phải là từ điển

            rating = comment.get("rating_star", "N/A")
            text = comment.get("comment", "N/A")
            order_id = comment.get("orderid", "N/A")
            create_time = comment.get("ctime", "N/A")  # Lấy thêm trường time
            if create_time:
                time = datetime.utcfromtimestamp(create_time).strftime('%Y-%m-%d %H:%M:%S')
            else:
                time = "N/A"


            writer.writerow([rating, text, order_id, time])

    print(f"Dữ liệu đã được lưu vào {file_path}")

def fetch_all_comments(item_id, shop_id, limit=10, max_pages=5):
    """
    Lấy tất cả bình luận bằng cách phân trang, giới hạn số trang
    """
    all_comments = []
    offset = 0
    pages_fetched = 0

    while True:
        if pages_fetched >= max_pages:  # Dừng nếu đã đạt giới hạn số trang
            print(f"Đã lấy đủ {max_pages} trang bình luận.")
            break

        comments = fetch_shopee_comments(item_id, shop_id, limit=limit, offset=offset)
        if not comments:  # Nếu không còn bình luận, dừng
            print("Không còn bình luận nào.")
            break

        # Kiểm tra dữ liệu trả về từ API
        if not isinstance(comments, list):
            print(f"Unexpected response format: {comments}")
            break

        all_comments.extend(comments)
        offset += limit  # Chuyển sang trang tiếp theo
        pages_fetched += 1
        print(f"Đã lấy {len(all_comments)} bình luận từ {pages_fetched} trang...")

    return all_comments

# Main
if __name__ == "__main__":
    # ID sản phẩm và shop từ URL Shopee
    item_id = 22088583698  # ID sản phẩm
    shop_id = 196261835    # ID shop

    # Giới hạn số trang cần lấy
    max_pages = 3  # Ví dụ: Lấy tối đa 3 trang

    # Lấy tất cả bình luận
    comments = fetch_all_comments(item_id, shop_id, limit=50, max_pages=max_pages)  # Lấy 50 bình luận mỗi trang

    # Tùy chỉnh đường dẫn lưu file CSV
    file_path = "..\data\shopee_data_raw.csv"
    save_comments_to_csv(comments, file_path=file_path)

Đã lấy 50 bình luận từ 1 trang...
Đã lấy 100 bình luận từ 2 trang...
Đã lấy 150 bình luận từ 3 trang...
Đã lấy đủ 3 trang bình luận.
Dữ liệu đã được lưu vào ..\data\shopee_data_raw.csv
